# Setup

In [ ]:
# Import necessary packages
from selenium import webdriver
import pandas as pd
import time
from datetime import datetime

# Google review url(s)
url = "https://www.google.com/maps/place/Nespresso/@49.2258269,-123.0014231,17z/data=!4m13!1m7!3m6!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!2sNespresso!8m2!3d49.2258269!4d-122.9992344!10e1!3m4!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!8m2!3d49.2258269!4d-122.9992344";

url_review = "https://www.google.com/maps/place/Nespresso/@49.2258269,-123.0014231,17z/data=!3m1!5s0x54867659512e6deb:0x8a75309114df0b12!4m15!1m7!3m6!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!2sNespresso!8m2!3d49.2258269!4d-122.9992344!10e1!3m6!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!8m2!3d49.2258269!4d-122.9992344!9m1!1b1";

# Run Web Scraper

In [ ]:
# Instantiate web driver
driver = webdriver.Chrome("chromedriver");
driver.set_window_size(1024, 600);
driver.maximize_window();

In [ ]:
# Go to Google Reviews URL 
driver.get(url);
time.sleep(3);

In [ ]:
# Get the total number of Google reviews and overall Google review rating
def get_generalMetrics(xpath):
    driver.find_element("xpath", xpath).location_once_scrolled_into_view;
    time.sleep(1);
    metrics = driver.find_element("xpath", xpath).text.split("\n");
    data = [[metrics[0], metrics[1].split(" ")[0]]];
    df_generalMetrics = pd.DataFrame(data, columns=['Overall Rating','Total Number of Reviews']);
    return df_generalMetrics;

get_generalMetrics('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[32]/div/div[2]')

In [ ]:
# Distribution of stars given
def get_reviewStarDistribution(parent_xpath):
    # driver.find_element("xpath", parent_xpath).location_once_scrolled_into_view;
    data = [];
    for i in range(1,6):
        label = driver.find_element("xpath", parent_xpath+'tr['+str(i)+']').get_attribute("aria-label").split(", ");
        time.sleep(1);
        data.append([label[0].split(" ")[0], label[1].split(" ")[0]]);
    df_reviewStar = pd.DataFrame(data, columns=['Stars Given','Number of Reviews']);
    return df_reviewStar;

get_reviewStarDistribution('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[32]/div/div[1]/table/tbody/')

In [ ]:
# Access full list of Google reviews
driver.get(url_review);
time.sleep(3);

In [ ]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

# Shutdown Web Driver

In [ ]:
driver.close()